In [1]:
# Recognise Faces using some classification algorithm like Logistic, KNN, SVM etc.

# 1. load the training data (numpy arrays of all the persons) 
           # x- values are stored in the numpy arrays
           # y-values we need to assign for each perso
# 2. Read a video stream using opencv
# 3. extract faces out of it
# 4. use knn to find the prediction of face (int)
# 5. map the predicted id to name of the user
# 6. Display the predictions on the screen - bounding box and name

In [1]:
import cv2
import numpy as np 
import os 

In [2]:
########## KNN CODE ############
def distance(p1,p2) :
    return ((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)**0.5

def knn(train, test, k=5):
	dist = []
	
	for i in range(train.shape[0]):
		# Get the vector and label
		ix = train[i, :-1] #rest of the coloumn for feauture part
		iy = train[i, -1]  #last coloumn for lable part
		# Compute the distance from test point
		d = distance(test, ix)
		dist.append([d, iy])
	# Sort based on distance and get top k
	dk = sorted(dist, key=lambda x: x[0])[:k]
	# Retrieve only the labels
	labels = np.array(dk)[:, -1]
	
	# Get frequencies of each label
	output = np.unique(labels, return_counts=True)
	# Find max frequency and corresponding label
	index = np.argmax(output[1])
	return output[0][index]
################################

In [15]:
#initialize camera
cap=cv2.VideoCapture(0)
#detect faces
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

skip=0

dataset_path='./empty/'
face_data=[]
labels=[]

class_id=0 #Labels for the given file
names={} # Mapping bte id-name

#data prepration
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4] #create a mapping b/w class_id and name
        print("Loaded"+fx)
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #create labels for the class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)

face_data
face_dataset=np.concatenate(face_data,axis=0)
face_lables=np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_lables.shape)

trainset=np.concatenate((face_dataset,face_lables),axis=1)
print(trainset.shape)
#*********DATASET READY**********

#TESTING PART

while True:
    ret,frame =cap.read()
    if ret==False:
        continue
        
    faces = face_cascade.detectMultiScale(frame,1.3,5) 
    
    for face in faces[-1:]:
        x,y,w,h=face
        
        #get the ace ROI
        offset=10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        
        #predicted lable 
        out=knn(trainset,face_section.flatten())
        
        #Display on screen the name and rectangle around it
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
    cv2.imshow("Faces",frame)
    
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release() #release web cam
cv2.destroyAllWindows()

Loadedanusha.npy
LoadedAryan_Mittal.npy
Loadedbacha.npy
Loadedvandana.npy
(46, 30000)
(46, 1)
(46, 30001)
